<font size="6">Fractional Processes & Power Spectral Density</font>

The goal of this notebook is to make a maximum likelihood estimator that takes into account the auto-correlation of the data, not just mean and variance or other statistical moments. The starting point for that would seem to be some kind of distance metric, but this is fundamentally what I need help with or to know if it's even possible. 

Even if two sets of data have the same distributional parameters the order of the data in terms of variance or some other auto-correlated parameter may not be the same. This is the auto-correlation decay in the time domain. 

Processes with long memory or periodic memory will not be ergodic especially if the auto-correlation decay is slow and follows a power law. So is it possible to have a maximum likelihood estimator that would compute likelihoods for the mean, variance, and auto-covariance of the data considered all together?

<font size="5">RBF Kernel</font>

Inspired partially by the work of these researchers,
http://cermics.enpc.fr/~guyon/documents/VolatilityIsMostlyPathDependent_Slides_RiO2022_22Aug2022.pdf

In this paper they discuss how kernels can encode the long or short-term memory of the volatility dynamics. I have some other papers that discuss similar ideas. 

<font size="5">Use Case in Factor Analysis</font>

Can Factor Investing Become Scientific? Endogenous Volatility. 

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4205613

Irrational Capital is a good example of a research/fund looking at deep ESG metrics, 
https://www.irrational.capital/

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

import sklearn 
import sklearn.mixture as mix 

In [ ]:
assetlist = [ 'IEF', 'GSG', 'IXN' ]
num_components = 4

Create the subset of assets. We end up with a 3-dimensional multivariate gaussian system. 

In [ ]:
m6 = pd.read_csv('./data/assets.csv')
m6_assets = pd.DataFrame()

for sym in assetlist: 
    m6_assets[sym] = m6[m6['symbol'] == sym]['price'].values

<font size="5">Z-Scoring & Auto-Correlation Decay Plots</font>

Formatting the training data. There is only training data and no test data in this exercise (it is a purely descriptive exercise). Z-scored measurements of the log price for each asset based on a rolling window. At timepoint 't' the window looks back 'w' steps to calculate mu and sigma. 

$$\huge z_t = \frac{x_t - \mu_{t-w:t}}{\sigma_{t-w:t}}$$

There are regions where the current log price is either above or below its running average (as measured in units of standard deviation) and this formatting of the data preserves long memory based on looking at the auto-correlation decay plots. <b>It is important that the running average never takes into account future data which would introduce lookahead bias, so a rolling metric based on the list comprehension is used.</b> 

(Hurst Exponent Auto-Covariance Function) 

$$\huge cov(s,t) = \frac{1}{2} \left( t^{2H} + s^{2H} - |t-s|^{2H} \right)$$

In [ ]:
import scipy.stats as stats
import scipy 

m6_subset1 = m6_assets.copy()

# Window length 5 days
wlen = 45

# First convert to log price 
m6_subset1 = m6_subset1.apply(np.log)

# Clean data
m6_subset1 = m6_subset1.dropna().reset_index().drop(columns='index')

# Apply z-score in a rolling way that does not create lookahead bias 
rolling_zscore = lambda serie: [ stats.zscore(serie[x-wlen:x]).values[-1] for x in range(wlen, len(serie) + 1) ]

m6_subset = pd.DataFrame()
m6_subset['IEF'] = m6_subset1['IEF'].diff().dropna()
m6_subset['GSG'] = m6_subset1['GSG'].diff().dropna()
m6_subset['IXN'] = m6_subset1['IXN'].diff().dropna()

#m6_subset['IEF'] = rolling_zscore(m6_subset1['IEF'])
#m6_subset['GSG'] = rolling_zscore(m6_subset1['GSG'])
#m6_subset['IXN'] = rolling_zscore(m6_subset1['IXN'])

# View Data
sns.lineplot(data=m6_subset)
plt.show() 

# View Auto Correlation
decay_func = lambda S, _range: [S.autocorr(lag=Lag) for Lag in _range]
sns.lineplot(data=decay_func(m6_subset['IEF'], range(0, 100)), label='autocorr_IEF', marker='o') 
sns.lineplot(data=decay_func(m6_subset['GSG'], range(0, 100)), label='autocorr_GSG', marker='o')
sns.lineplot(data=decay_func(m6_subset['IXN'], range(0, 100)), label='autocorr_IXN', marker='o')
plt.xlabel('Lag')
plt.ylabel('auto-correlation')
plt.show()

%pip install torch
%pip install gpytorch
%pip install git+https://github.com/forrestbao/pyeeg.git
import torch
import gpytorch
import pyeeg

class FractionalPSD:
    def __init__(self, data):
        self.data = np.array(data)

    def distance_norm(self, target): 
        x1 = torch.tensor([ _x for _x in target ], dtype=torch.float)
        x0 = torch.tensor([ _x for _x in self.data  ], dtype=torch.float)
        
        # Evaluate kernel auto-covariance matrix 
        kernel = gpytorch.kernels.RBFKernel()
        autocov_1 = (kernel(x1).evaluate()).detach().numpy() 
        autocov_0 = (kernel(x0).evaluate()).detach().numpy()

        # Frobius distance between kernel evaluations 
        return np.linalg.norm(autocov_1 - autocov_0, 'fro') 
    
# Example comparing white noise with same mean/variance to fractional processes 
fractional_ief = FractionalPSD(m6_subset['IEF'].values)
fractional_gsg = FractionalPSD(m6_subset['GSG'].values)
fractional_ixn = FractionalPSD(m6_subset['IXN'].values)
scores_ief=[]
scores_gsg=[]
scores_ixn=[]
scores=[]
hursts=[]

white_noise = torch.normal(np.mean(fractional_ief.data), np.var(fractional_ief.data), size=(len(m6_subset),))
white_noise = white_noise.detach().numpy()
hursts.append(pyeeg.hurst(white_noise))
white_noise = FractionalPSD(white_noise)

scores_ief.append(white_noise.distance_norm(fractional_ief.data))
scores_gsg.append(white_noise.distance_norm(fractional_gsg.data))
scores_ixn.append(white_noise.distance_norm(fractional_ixn.data))
print(scores_ief)
print(scores_gsg)

plt.scatter(x=range(len(scores_ief)), y=scores_ief, s=8, label='scores_ief')
plt.scatter(x=range(len(scores_gsg)), y=scores_gsg, s=8, label='scores_gsg')
plt.scatter(x=range(len(scores_ixn)), y=scores_ixn, s=8, label='scores_ixn')
plt.legend()
plt.show()

Instantiate a non-parametric model using the dirichlet process and view the clustering results on the training data. The shaded regions on the plot represent the final state the system lands on, states[-1]. 

In [ ]:
dpgmm_model = mix.BayesianGaussianMixture(n_components=num_components, 
                                          weight_concentration_prior_type='dirichlet_process', 
                                          n_init=1, 
                                          max_iter=1000)

In [ ]:
dpgmm_model.fit(m6_subset)
states = dpgmm_model.predict(m6_subset)

# Count the frequency of each latent state 
state_counts = np.zeros(num_components)
for M in states:
  state_counts[M] += 1 

<font size=5> Precision Matrices as Regime-Specific Graphs </font>

The precision matrix is just the inverse of the covariance matrix. Connections between nodes representing degree of dependence/independence within each respective latent state regime. Nodes are one dimension (feature) of the data. 

In [ ]:
%pip install networkx 
import networkx as nx
import matplotlib.pyplot as plt 

fig, axes = plt.subplots(2, 2, figsize=(8, 5))

i = 0
j = 0 
for pmat in dpgmm_model.precisions_:
    if state_counts[i] > 0: 
        ax = axes[j//2][j%2]
        G = nx.DiGraph(pmat)
        nx.draw(G, with_labels=True, ax=ax)
        j += 1
    i += 1

plt.show()